#### Load dependencies

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, SpatialDropout1D, LSTM
from keras.layers.wrappers import Bidirectional
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
output_dir = 'model_output/biLSTM'

In [4]:
!ls

alexnet_in_keras.ipynb
bidirectional_lstm.ipynb
bidirectional_lstm_myself.ipynb
cartpole_dqn.ipynb
clean_gutenberg_tsne.csv
convolutional_sentiment_classifier.ipynb
creating_word_vectors_with_word2vec.ipynb
cross_entropy_cost.ipynb
deep_net_in_keras.ipynb
deep_net_in_tensorflow.ipynb
dense_sentiment_classifier.ipynb
first_tensorflow_graphs.ipynb
first_tensorflow_neurons.ipynb
generative_adversarial_network.ipynb
gru_in_keras.ipynb
intermediate_net_in_keras.ipynb
intermediate_net_in_tensorflow.ipynb
intro_to_tensorflow_times_a_million.ipynb
lenet_in_keras.ipynb
lenet_in_tensorflow.ipynb
live_training
multi_convnet_architectures.ipynb
natural_language_preprocessing_best_practices.ipynb
point_by_point_intro_to_tensorflow.ipynb
rnn_in_keras.ipynb
shallow_net_in_keras.ipynb
sigmoid_function.ipynb
softmax_demo.ipynb
stacked_bidirectional_lstm.ipynb
tensor-fied_intro_to_tensorflow.ipynb
vanilla_lstm_in_keras.ipynb
vggnet_in_keras.ipynb
ye_olde_conv_lstm_stackeroo.ipynb


In [21]:
epochs = 1
batch_size = 128

In [22]:
n_dim = 64
n_unique_words = 10000
max_review_length = 200
pad_type = trunc_type = 'pre'
drop_embed = 0.2 

n_lstm = 256
drop_lstm = 0.2

#### Loading data

In [23]:
(X_train, y_train), (X_valid, y_valid) = imdb.load_data(num_words=n_unique_words)

#### Preprocessing data

In [24]:
X_train = pad_sequences(X_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
X_valid = pad_sequences(X_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

#### Designing neural network architecture

In [25]:
model = Sequential()

In [26]:
model.add(Embedding(n_unique_words, n_dim, input_length=max_review_length))
model.add(SpatialDropout1D(drop_embed))
model.add(Bidirectional(LSTM(n_lstm, dropout=drop_lstm)))
model.add(Dense(1, activation='sigmoid'))

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 64)           640000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 200, 64)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               657408    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
Total params: 1,297,921
Trainable params: 1,297,921
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [16]:
X_valid

array([[   0,    0,    0, ..., 2305,   18,   12],
       [  26, 1200,  496, ...,   17,   25,   70],
       [   9, 1868, 3996, ...,  483,   10,   10],
       ..., 
       [   2, 2086,    9, ...,    4, 3769, 3724],
       [  76,  662,    5, ...,   11,    4, 4039],
       [   0,    0,    0, ...,   11,    4,  130]], dtype=int32)

In [17]:
X_valid.ndim, X_valid.shape, X_valid.dtype

(2, (25000, 200), dtype('int32'))

In [29]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")

In [30]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, \
          validation_data=(X_valid, y_valid), callbacks=[modelcheckpoint])

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
12672/25000 [==============>...............] - ETA: 1426s - loss: 0.6723 - acc: 0.6029